In [6]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
import collections
import pickle
import numpy as np
from datetime import datetime

In [13]:
df = pd.read_csv(f"/home/marcin/PycharmProjects/Engineering_Thesis/data_provider/data/CriteoSearchData-sorted-no-duplicates-LabelEncoded.csv"
                 ,sep=','
                ,nrows=5000000
#                 ,skiprows=range(1,2000000)
                )

In [7]:
y_t = [0,1,1,0,0,0,0,1,0,0]
w_array = np.array([1.1]*len(y_t))
for id, value in enumerate(y_t):
    if value == 0:
        w_array[id] = 2.1
    else:
        w_array[id] = 3.1
w_array

array([2.1, 3.1, 3.1, 2.1, 2.1, 2.1, 2.1, 3.1, 2.1, 2.1])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif 
X = df.loc[:,~df.columns.isin(['sale','click_timestamp','sales_amount_in_euro','time_delay_for_conversion','product_price'])]
y = df['sale']
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=f_classif, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

In [ ]:
# df = df.rename(columns={'Sale':'sale','SalesAmountInEuro':'sales_amount_in_euro'})

In [ ]:
df[:3]

In [ ]:
# def convert_timestamp(df):
#     temp = datetime.fromtimestamp(df['click_timestamp'])
#     df['year'] = temp.year
#     df['month'] = temp.month
#     df['day'] = temp.day
#     df['hour'] = temp.hour
#     return df

# df = df.apply(convert_timestamp, axis=1)
# df = df.drop(columns=['click_timestamp'], axis=1)

In [ ]:
df[:3]

In [ ]:
# names_of_columns_with_ids = ['audience_id', 'device_type',
#        'partner_id', 'product_age_group', 'product_brand',
#        'product_category_1', 'product_category_2', 'product_category_3',
#        'product_category_4', 'product_category_5', 'product_category_6',
#        'product_category_7', 'product_country', 'product_gender', 'product_id',
#        'product_title', 'user_id']

In [ ]:
# utworzenia słownika LabelEncoders dla całego zbioru danych
# LabelEncoders_dict = collections.defaultdict(LabelEncoder)


# df[names_of_columns_with_ids] = df[names_of_columns_with_ids].astype(str)
# # Encoding the variable
# df[names_of_columns_with_ids] = df[names_of_columns_with_ids].apply(lambda x: LabelEncoders_dict[x.name].fit_transform(x))

In [ ]:
# file = open(
#     f"/home/marcin/PycharmProjects/Engineering_Thesis/build_and_update_model_server/LabelEncoders_dict2.pickle",
#     "wb")
# LabelEncoders_dict = pickle.load(file)

In [ ]:
# file = open(
#     f"/home/marcin/PycharmProjects/Engineering_Thesis/build_and_update_model_server/LabelEncoders_dict2.pickle",
#     "rb")
# LabelEncoders_dict = pickle.load(file)

In [11]:
from xgboost import XGBClassifier

def first_model(x_train, y_train):
#     df[names_of_columns_with_ids] = df[names_of_columns_with_ids].astype(str)
#     df[names_of_columns_with_ids] = df[names_of_columns_with_ids].apply(lambda x: LabelEncoders_dict[x.name].transform(x))
#     df[names_of_columns_with_ids] = df[names_of_columns_with_ids].apply(LabelEncoder().fit_transform)

    
    X_train, X_test, y_train, y_test = train_test_split(x_train, 
                                                    y_train,
                                                    test_size=0.2,
                                                    random_state=0)
    number_of_samples = len(y_train)
    counter = collections.Counter(y_train)
    percent_of_ones = counter[1]/number_of_samples
    
    w_array = np.array([1.1]*y_train.shape[0])
    for id, value in enumerate(y_train):
        if value == 0:
            w_array[id] = percent_of_ones
        else:
            w_array[id] = 1-percent_of_ones
#     model =  SGDClassifier(loss='log', random_state=1, tol=1e-3, max_iter=1000, penalty='l1', alpha=1e-05, n_jobs=-1, class_weight={0:percent_of_ones, 1:1-percent_of_ones})
    model = XGBClassifier()
    model.fit(X_train, y_train, sample_weight=w_array)
    
    y_pred = model.predict(X_test)
    

    print(collections.Counter(y_test))
    print(collections.Counter(y_pred))

    print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
    print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
    print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
    print('classification_report :\n', classification_report(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print(confmat)
    
    return model, X_test, y_test
    

In [ ]:
def next_models(model, df, LabelEncoders_dict, X_test, y_test):
    df[names_of_columns_with_ids] = df[names_of_columns_with_ids].astype(str)
    df[names_of_columns_with_ids] = df[names_of_columns_with_ids].apply(lambda x: LabelEncoders_dict[x.name].transform(x))
    
    
    model = model.partial_fit(df.loc[:,df.columns != 'sale'], df['sale'])
    
    y_pred = model.predict(X_test)

    print(collections.Counter(y_test))
    print(collections.Counter(y_pred))

    print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
    print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
    print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
    print('classification_report :\n', classification_report(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print(confmat)
    return model

In [14]:
# df = pd.read_csv(f"/home/marcin/PycharmProjects/Engineering_Thesis/build_and_update_model_server/test1.csv"
#                  ,sep=','
#                 )
# x_train = df.loc[:,~df.columns.isin(['sale','click_timestamp','sales_amount_in_euro','time_delay_for_conversion'])]
x_train = df[['device_type','audience_id','partner_id','product_brand','nb_clicks_1week','product_gender','product_age_group']]
y_train = df['sale']
model, X_test, y_test = first_model(x_train, y_train)

Counter({0: 884943, 1: 115057})
Counter({0: 611430, 1: 388570})
balanced_accuracy_score: 0.6612829532811477
accuracy_score: 0.651475
Nieprawidłowo sklasyfikowane próbki: 348525
classification_report :
               precision    recall  f1-score   support

           0       0.94      0.65      0.77    884943
           1       0.20      0.67      0.31    115057

    accuracy                           0.65   1000000
   macro avg       0.57      0.66      0.54   1000000
weighted avg       0.85      0.65      0.71   1000000

[[573924 311019]
 [ 37506  77551]]


In [ ]:
df = pd.read_csv(f"/home/marcin/PycharmProjects/Engineering_Thesis/build_and_update_model_server/test2.csv"
                 ,sep=','
                )
x_train = df.loc[:,~df.columns.isin(['sale'])]
y_train = df['sale']
model, X_test, y_test = first_model(x_train, y_train)

In [ ]:
print("PIERWSZY MODEL")
x_train = df.loc[:,~df.columns.isin(['sale','sales_amount_in_euro','time_delay_for_conversion','product_price'])]
y_train = df['sale']
model, X_test, y_test = first_model(x_train, y_train)

In [ ]:
print("PIERWSZY MODEL")
x_train = df.loc[:,~df.columns.isin(['sale'])]
y_train = df['sale']
model, X_test, y_test = first_model(x_train, y_train)


# for i in range(1, 8):
#     print("=========================================")
#     print("MODEL NUMER: ", i+1)
#     df2 = pd.read_csv(f"/home/marcin/PycharmProjects/Engineering_Thesis/data_provider/data/CriteoSearchData-sorted-no-duplicates.csv""
#                  ,sep='\t'
#                 ,nrows=1000000
#                 ,skiprows=range(1, 2000000*i)
#                 )
#     model = next_models(model, df2, LabelEncoders_dict, X_test, y_test)


In [ ]:
df3 = pd.read_csv(f"/home/marcin/PycharmProjects/Engineering_Thesis/data_provider/data/CriteoSearchData-sorted-no-duplicates-LabelEncoded.csv"
                 ,sep=','
                ,nrows=1000000
                ,skiprows=range(1,2000000)
                )

In [ ]:
df3 = df3.rename(columns={'Sale':'sale','SalesAmountInEuro':'sales_amount_in_euro'})

In [ ]:
import json
import time

s = time.time()
for id, row in df[:10000].iterrows():
    sample_json = row.to_json()
#     s = time.time()
    sample_dict = json.loads(sample_json)    
    sample_dict_result = sample_dict.copy()
    sample_dict.pop('sale', None)
    probabilities = model.predict_proba([list(sample_dict.values())])[0].ravel()
    if probabilities[0] > probabilities[1]:
        sample_dict_result['predicted'] = 0
    else:
        sample_dict_result['predicted'] = 1
        
    sample_dict_result['probabilities'] = json.dumps(list(probabilities))
#     model.predict([list(sample_dict.values())])

#     print(time.time()-s)
#     break
    
print(time.time()-s)


In [ ]:
df2 = pd.read_csv(f"/home/marcin/PycharmProjects/Engineering_Thesis/data_provider/data/CriteoSearchData-sorted-no-duplicates.csv"
             ,sep='\t'
             ,nrows=1000
            )
df2[names_of_columns_with_ids] = df2[names_of_columns_with_ids].astype(str)
s2 = time.time()
for index, value in LabelEncoders_dict.items():
    s = time.time()
    LabelEncoders_dict[index].transform(df2[index])
    print(index)
    print(time.time()-s)
print(time.time()-s2)

In [ ]:
df['product_category(7)'] = df['product_category(7)'].astype(str)
df['product_title'] = df['product_title'].astype(str)

In [ ]:
df.iloc[:,6:] = df.iloc[:,6:].apply(LabelEncoder().fit_transform)
df[:3]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], 
                                                    df['Sale'],
                                                    test_size=0.2,
                                                    random_state=0)

In [ ]:
number_of_samples = len(y_train)
counter = collections.Counter(y_train)
percent_of_ones = counter[1]/number_of_samples
percent_of_ones

In [ ]:
# model =  SGDClassifier(loss='log', random_state=1, tol=1e-3, max_iter=1000, penalty='l1', alpha=1e-05, n_jobs=-1)
model =  SGDClassifier(loss='log', random_state=1, tol=1e-3, max_iter=1000, penalty='l1', alpha=1e-05, n_jobs=-1, class_weight={0:percent_of_ones, 1:1-percent_of_ones})
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
import collections
print(collections.Counter(y_test))
print(collections.Counter(y_pred))

print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('classification_report :\n', classification_report(y_test, y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)